In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import pandas as pd
import numpy as np 
import pickle

from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList
import matplotlib.pyplot as plt
from codes.models.Naive.Naive_utils import get_pred_naive, get_pred_sentence, get_risk_pheno
from functools import partial

In [ ]:
file = '/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Training/SNPS/1/rs673604/Abby/dic_data_rs673604_decorelate=False.pkl'
with open(file, 'rb') as file:
    o = pickle.load(file)

In [ ]:
names_pheno = ''
ids_pheno = [374]

In [ ]:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = True
save_data = False
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 1
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = True
list_env_features = ['age', 'sex']
### data format
batch_size = 20
data_share = 1

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=True,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share)
#patient_list = dataT.get_patientlist()


In [ ]:
dic = dataT.get_data_tabtransfo()

In [ ]:
data, labels, indices_env, name_envs = dataT.get_tree_data(with_env=False)

In [ ]:
counts = np.sum(data, axis=0)
nb_phenos = data.shape[1]
phenos = np.arange(0, nb_phenos)
frequencies = counts/len(data)

In [ ]:
def get_F0_F1(data_tree, labels, id):
    phenos_zero = data[:, data[id]==0]
    phenos_ones = data[:, data[id]==1]
    labels_zeros = labels[data[id]==0]
    labels_ones = labels[data[id]==1]

    P0 = np.sum(labels_zeros) / len(labels_zeros)
    P1 = np.sum(labels_ones) / len(labels_ones)

    F0 = max(P0, 1-P0)
    F1 = max(P1, 1-P1)

    return P0, P1, F0, F1




In [ ]:
id = 4316

In [ ]:
odds_ratios[id]

In [ ]:
np.where((odds_ratios>0) & (counts>1000))

In [ ]:
frequencies

In [ ]:
plt.plot(odds_ratios, 'o')

In [ ]:
mean_pop = np.sum(labels) / len(labels)

In [ ]:
mean_pop

In [ ]:
id = 44

In [ ]:
counts[id], frequencies[id]

In [ ]:
labels_pheno = labels[data[:, id]==1]

In [ ]:
np.sum(labels_pheno==1)/ len(labels_pheno)

In [ ]:
len(data)

In [ ]:
plt.plot(labels[data[:, 44]==1])

In [ ]:
np.sum(labels_pheno==1)/ len(labels_pheno)

In [ ]:
file = "/gpfs/commons/datasets/controlled/ukbb-gursoylab/"+f"omop_processed/ukbb_omop_rolled_up_depth_{4}_closest_ancestor.csv"
pheno_df_paul = pd.read_csv(file)

In [ ]:
grouped = pheno_df_paul.groupby('concept_id')

In [ ]:
df = grouped.get_group(432867)

In [ ]:
eids = np.array(df.eid)

In [ ]:
eids.shape, np.unique(eids).shape

In [ ]:
nb_phenos = np.max(pheno_df['concept_id'])
res = np.zeros((len(eid_list), nb_phenos), dtype=int)

grouped = pheno_df.groupby('eid')

data_list  = [np.array(grouped.get_group(eid)['concept_id'].unique())
            for eid in eid_list]

for k, diseases in enumerate(data_list):
    res[k,np.array(diseases)-1] = 1
nb_max_diseases_sentence = max([len(sentence) for sentence in data_list])
columns = np.arange(1, nb_phenos+1)
pheno_df = pd.DataFrame(data=res, columns=columns)
pheno_df.insert(0, 'eid', eid_list)

env_df = pd.read_csv(self.env_file)
env_df.set_index('eid', inplace=True)
pheno_df.set_index('eid', inplace=True)
list_features_env = self.list_env_features if with_env else []
df_tot = pheno_df.join(env_df[list_features_env], on=pheno_df.index, rsuffix='')
